# Final Project - Part 2
## Billionaire Dataset Dashboard

Date: 04/12/2019

In [1]:
import pandas as pd
import numpy as np
import bqplot
from bqplot import pyplot as plt
from bqplot import *
import bqplot.market_map
import ipywidgets
from ipywidgets import Layout

In [2]:
data = pd.read_csv('billionaires.csv')

In [3]:
data.head()

,age,category,citizenship,company.name,company.type,country code,founded,from emerging,gdp,gender,...,name,rank,region,relationship,sector,was founder,was political,wealth.type,worth in billions,year
0,-1,Financial,Saudi Arabia,Rolaco Trading and Contracting Company,new,SAU,1968,True,1.580000e+11,male,...,Abdul Aziz Al-Sulaiman,404,Middle East/North Africa,founder,construction,True,False,self-made finance,1.0,1996
1,34,Financial,United States,Fidelity Investments,new,USA,1946,True,8.100000e+12,female,...,Abigail Johnson,145,North America,relation,investment banking,True,False,inherited,2.5,1996
2,59,Non-Traded Sectors,Brazil,Companhia Brasileira de Distribui?ao,new,BRA,1948,True,8.540000e+11,male,...,Abilio dos Santos Diniz,322,Latin America,relation,retail,True,False,inherited,1.2,1996
3,61,New Sectors,Germany,Ratiopharm,new,DEU,1881,True,2.500000e+12,male,...,Adolf Merckle,388,Europe,relation,pharmaceuticals,True,False,inherited,1.0,1996
4,-1,Financial,Hong Kong,Swire,new,HKG,1816,True,1.600000e+11,male,...,Adrian and John Swire,162,East Asia,relation,trading company,True,False,inherited,2.2,1996


In [4]:
heat_piv = data.pivot_table(values='country code', index='citizenship', 
                              columns='year', aggfunc='count').fillna(0).reset_index()

In [5]:
heat_piv.head()

year,citizenship,1996,2001,2014
0,Algeria,0.0,0.0,1.0
1,Angola,0.0,0.0,1.0
2,Argentina,3.0,4.0,5.0
3,Australia,1.0,3.0,29.0
4,Austria,0.0,0.0,10.0


In [6]:
len(heat_piv['citizenship'])

73

## Create Year Dropdown

In [7]:
year_list = list(data['year'].unique())

In [8]:
#create a year dropdown  
year_dropdown = ipywidgets.widgets.Dropdown(
                        options=year_list,
                        value=year_list[0],
                        description='Year:',
                        disabled=False
                    )

## Create Country Market Map

In [9]:
def mmap_data(piv_data, data, year):
    mmap_piv = piv_data[['citizenship', year]]
    region = data[['citizenship', 'region']].drop_duplicates().sort_values('citizenship')
    result = pd.merge(mmap_piv, region, how='left', on=['citizenship'])
    
    
    num_of_bill = result[year].replace({0:np.nan})
    country = result['citizenship']
    region = result['region']
    
    return num_of_bill, country, region

In [10]:
num_of_bill, country, region = mmap_data(heat_piv, data, year_dropdown.value) 
# num_of_bill, country = mmap_data(heat_piv, year_dropdown.value) 

In [11]:
# create a color map 
c_sc_mm = bqplot.ColorScale(scheme = "RdPu")
c_ax_mm = bqplot.ColorAxis(scale = c_sc_mm, orientation = 'vertical', label='Number of Billionaires')

#want to sort group by regions or continents and show with different colors
mmap = bqplot.market_map.MarketMap(color = num_of_bill, 
                                  names = country,
                                  groups = region,
                                  colors = ['Grey'],
                                  scales={'color':c_sc_mm}, axes=[c_ax_mm],
                                  title='Billionaire Count Market Map',
                                  enable_select=True)

## Create GDP World Map

In [12]:
world_map_piv = data.pivot_table(values='gdp', index='citizenship', 
                              columns='year', aggfunc=lambda x: sum(x.unique())).fillna(0).reset_index() 

In [13]:
world_map_piv.head()

year,citizenship,1996,2001,2014
0,Algeria,0.000000e+00,0.000000e+00,0.0
1,Angola,0.000000e+00,0.000000e+00,0.0
2,Argentina,2.720000e+11,2.690000e+11,0.0
3,Australia,4.010000e+11,3.790000e+11,0.0
4,Austria,0.000000e+00,0.000000e+00,0.0


In [14]:
# get country id from "WorldMap.json" file
default_map = Map(map_data=topo_load('map_data/WorldMap.json'))
map_prop_list = default_map.map_data['objects']['subunits']['geometries']
map_id_dict = {}
for i in range(len(map_prop_list)):
    map_id_dict[map_prop_list[i]['properties']['name']] = map_prop_list[i]['id']

In [15]:
# change the country name that is different from the WorldMap.json
map_id_dict['Russia'] = map_id_dict.pop('Russian Federation')
map_id_dict['Taiwan'] = map_id_dict.pop('Taiwan, Province of China')

In [16]:
def create_gdp_colormap(piv_data, year, country_id_dict):
    gdp = dict(zip(piv_data['citizenship'], piv_data[year].values.astype(int)))
    
    country_id_keys = list(country_id_dict)
    gdp_keys = list(gdp)
    color_dict = {}
    for i in range(len(country_id_keys)):
        if country_id_keys[i] in gdp_keys:
            color_dict[country_id_dict[country_id_keys[i]]] = gdp[country_id_keys[i]]
        else:
            color_dict[country_id_dict[country_id_keys[i]]] = None
        
    return color_dict

In [17]:
color_dict = create_gdp_colormap(world_map_piv, year_dropdown.value, map_id_dict)

In [18]:
len(color_dict)

177

In [19]:
map_tt = Tooltip(fields=['name', 'color'], labels=['Country', 'GDP'])

c_sc = ColorScale(scheme = "RdPu")
c_ax = ColorAxis(scale = c_sc, label='GDP')

bill_map = Map(map_data=topo_load('map_data/WorldMap.json'), tooltip=map_tt, 
               color = color_dict, 
               scales = {'projection': Mercator(), 'color': c_sc}, 
               colors = {'default_color': 'Grey'},
               selected = [] # update when market map is selected
              ) 

map_fig = Figure(marks=[bill_map], axes=[c_ax],title='Billionaire GDP World Map')

In [20]:
# create a selector for the heat map and world map
mySelectedLabel = ipywidgets.Label()

## Create Bar Chart - Industry vs Number of Billionaires

In [21]:
industry_piv = data.pivot_table(values='name', 
                                index='industry', 
                                columns=['year', 'citizenship'], 
                                aggfunc='count').fillna(0).reset_index().drop(index=0).set_index('industry')

In [22]:
industry_piv[2014]['Hong Kong']

industry
Constrution                         0.0
Consumer                            6.0
Diversified financial               5.0
Energy                              1.0
Hedge funds                         0.0
Media                               1.0
Mining and metals                   0.0
Money Management                    1.0
Non-consumer industrial             1.0
Other                               2.0
Private equity/leveraged buyout     0.0
Real Estate                        23.0
Retail, Restaurant                  3.0
Technology-Computer                 1.0
Technology-Medical                  1.0
Venture Capital                     0.0
banking                             0.0
services                            0.0
Name: Hong Kong, dtype: float64

In [23]:
# define the scales, we scale our x & y axis to the scales of the min and max of our data
x_sc_in = bqplot.OrdinalScale() 
y_sc_in = bqplot.LinearScale() 
x_ax_in = bqplot.Axis(scale = x_sc_in, label = 'Industry')
y_ax_in = bqplot.Axis(scale = y_sc_in, orientation = 'vertical', label='Number of Billionaires')

In [24]:
#create bar chart
select = pd.Series([]) 
bar = bqplot.Bars(x = [], y = [], scales={'x':x_sc_in, 'y':y_sc_in})
bar_fig = bqplot.Figure(marks = [bar], axes = [x_ax_in, y_ax_in], title='Number of Billionaires in each industry')

## Create Scatterplot - Founded Year vs Worth in Value
Differentiate the data with gender by color: male(blue), female(orange), married couple(green)

In [25]:
x_sc_sa = bqplot.OrdinalScale() 
y_sc_sa = bqplot.LinearScale() 
x_ax_sa = bqplot.Axis(scale = x_sc_sa, label = 'Company Founded Year')
y_ax_sa = bqplot.Axis(scale = y_sc_sa, orientation = 'vertical', label='Worth in billions')

In [26]:
c_sc_sa = bqplot.OrdinalColorScale(domain=data['gender'].unique().tolist(), 
                                colors=bqplot.CATEGORY10[:6])

#create scatterplot
scat = bqplot.Scatter(x = [], y = [], scales={'x':x_sc_sa, 'y':y_sc_sa, 'color':c_sc_sa}, color=data['gender'])
scat_fig = bqplot.Figure(marks = [scat], axes = [x_ax_sa, y_ax_sa], 
                         title='Company Founded Year vs Worth in Value'
                         )

## Create a Dashboard
1. Dashboard Elements
    1. Market Map (Controller) 
        - It controlls the world map, bar chart, and the scatterplot
        - Some grids in the market map do not show it is selected even if you click on the grid. This is an issue of the bqplot package that hasn't been solved
    2. World Map
        - It will turn red when the user selected a country from the Market Map 
        - It will show the name and gdp when the user points to the country 
        - The GDP that show 0 in some countries means either the country do not have billionaires in that year or missing values
    3. Bar Chart
        - It shows the number of billionaires of each industries based on the selected countries and year
    4. Scatter plot
        - It shows the relationship between founded year and worth in billions
        - The color of the data points represents the genders (male/female/married couple/nan)

2. Dataset issues 
    1. All the gdp records in 2014 and some gdp records in 2001 and 1996 are missing, which are shown as 0 in the world map tooltip
    2. GDP record of the countries that do not have billionaires are left blank
    3. This world map only contains 177 countries (but there are 192 countries in the world). Some countries such as Hong Kong, Macau, St. Kitts and Nevis which have billionaires aren't inluded in the WorldMap.json file. Thus, it could not be shown in the world map visualization even if we selected it on the market map.

### Dropdown / Market Map Option Change

In [27]:
# select year dropdown 
def year_changed(change):
    # year change, market map change
    mmap.color, mmap.name, region = mmap_data(heat_piv, data, year_dropdown.value) 
    # year change, world map change
    bill_map.selected = []
    bill_map.color = create_gdp_colormap(world_map_piv, year_dropdown.value, map_id_dict)

    try:
        ind_piv2 = pd.DataFrame(index=industry_piv.index)
        scat_piv2 = pd.DataFrame(index=data.index)
        map_select_list = []
        for i in range(len(mmap.selected)):
            if i == 0:
                # select the columns based on year and country
                try:
                    ind_piv2 = industry_piv[year_dropdown.value][mmap.selected[i]]
                    scat_piv2 = data[(data['year']==year_dropdown.value) & (data['citizenship']==mmap.selected[i])]
                except KeyError:
                    pass
            else:
                try: 
                    # select the columns based on year and country, then combine it with the previous dataframe
                    ind_piv2 = ind_piv2 + industry_piv[year_dropdown.value][mmap.selected[i]]
                    scat_piv = data[(data['year']==year_dropdown.value) & (data['citizenship']==mmap.selected[i])]
                    scat_piv2 = scat_piv2.append(scat_piv, sort=False) 
                except KeyError:
                    pass
                
            # ignore the countries that have billionaires but are not listed in WorldMap.json
            try:
                map_select_list.append(map_id_dict[mmap.selected[i]])
            except KeyError:
                pass      
                   
        # update bar chart
        bar.x = ind_piv2.index
        bar.y = ind_piv2.values
#         print(bar.y)
        x_sc_in.domain = list(ind_piv2.index)
        
        #update scatterplot
        sort_scat = scat_piv2.sort_values(by=['founded'])
        scat.x = sort_scat['founded']
        scat.y = sort_scat['worth in billions']
        x_sc_sa.domain = list(scat.x)

        # market map select: update color dict change
        bill_map.selected = map_select_list
        
    except KeyError:
        pass


year_dropdown.observe(year_changed, 'value')

In [28]:
# select market map country
def get_mmap_value(change):
    if mmap.selected == []: 
        bar.x = []
        bar.y = []
        x_sc_in.domain=[]
        scat.x = []
        bill_map.selected = [] 
        
    else: 
        try:
            ind_piv2 = pd.DataFrame(index=industry_piv.index)
            scat_piv2 = pd.DataFrame(index=data.index)
            map_select_list = []

            for i in range(len(mmap.selected)):
                if i == 0:
                    try:
                        # select the columns based on year and country
                        ind_piv2 = industry_piv[year_dropdown.value][mmap.selected[i]]
                        scat_piv2 = data[(data['year']==year_dropdown.value) & (data['citizenship']==mmap.selected[i])]
                    except KeyError:
                        pass
                else:
                    try:
                        # select the columns based on year and country, then combine it with the previous dataframe
                        ind_piv2 = ind_piv2 + industry_piv[year_dropdown.value][mmap.selected[i]]
                        scat_piv = data[(data['year']==year_dropdown.value) & (data['citizenship']==mmap.selected[i])]
                        scat_piv2 = scat_piv2.append(scat_piv, sort=False)  
                    except KeyError:
                        pass

                # ignore the countries that have billionaires but are not listed in WorldMap.json
                try:
                    map_select_list.append(map_id_dict[mmap.selected[i]])
                except KeyError:
                    pass
                
            
            bar.x = ind_piv2.index
            bar.y = ind_piv2.values
            x_sc_in.domain = list(ind_piv2.index)

            sort_scat = scat_piv2.sort_values(by=['founded'])
            scat.x = sort_scat['founded']
            scat.y = sort_scat['worth in billions']
            x_sc_sa.domain = list(scat.x)

            # market map select: update color dict change
            bill_map.selected = map_select_list
            
        except KeyError:
            pass
    print(mmap.selected) # check market map selection

mmap.observe(get_mmap_value, 'selected')

In [29]:
# For debugging
# When we execute the market map, a blue square will show if the country is selected
# However, some countries in market map such as Hong Kong and US does not show a blue square even if it is selected
# this cell is to see whether you have successfully selected the country or not 
mmap.selected

[]

In [30]:
mmap.layout.min_width = '960px'
mmap.layout.min_height = '400px'

map_fig.layout.min_width = '960px'
map_fig.layout.min_height = '600px'

ipywidgets.VBox([mySelectedLabel, 
                 ipywidgets.HBox([year_dropdown]), 
                 mmap, map_fig, 
                 ipywidgets.HBox([bar_fig, scat_fig])]) 

['Peru']
['Peru', 'Colombia']
['Peru', 'Colombia', 'Singapore']
['Peru', 'Colombia', 'Singapore', 'Finland']
